In [74]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [75]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')
df_test = pd.read_csv('../test_filled_new.csv')

In [76]:
df_train.columns,df_test.columns

(Index(['Unnamed: 0', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'Year', 'Month', 'Tavg_week', 'precip_week', 'heat_dw', 'cool_dw', '1_week_avrgPrecipTotal', '1_week_avrgTavg', '2_week_avrgPrecipTotal', '2_week_avrgTavg', '3_week_avrgPrecipTotal', '3_week_avrgTavg', '4_week_avrgPrecipTotal', '4_week_avrgTavg', '5_week_avrgPrecipTotal', '5_week_avrgTavg', '6_week_avrgPrecipTotal', '6_week_avrgTavg', '7_week_avrgPrecipTotal', '7_week_avrgTavg', '8_week_avrgPrecipTotal', '8_week_avrgTavg', '9_week_avrgPrecipTotal', '9_week_avrgTavg', '10_week_avrgPrecipTotal', '10_week_avrgTavg', '11_week_avrgPrecipTotal', '11_week_avrgTavg', '12_week_avrgPrecipTotal', '12_week_

In [77]:
[var for var in df_train.columns if var not in df_test.columns]

['Unnamed: 0', 'NumMosquitos', 'WnvPresent']

In [78]:
X = df_train
X.dtypes

Unnamed: 0                  int64
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
Station                     int64
Tmax                        int64
...
16_week_avrgTavg           float64
17_week_avrgPrecipTotal    float64
17_week_avrgTavg           float64
18_week_avrgPrecipTotal    float64
18_week_avrgTavg           float64
19_week_avrgPrecipTotal    float64
19_week_avrgTavg           float64
20_week_avrgPrecipTotal    float64
20_week_avrgTavg           float64
21_week_avrgPrecipTotal    float64
21_week_avrgTavg           float64
22_week_avrgPrecipTotal    float64
22_week_avrgTavg           float64
23_week_avrgPrecipTotal    floa

In [79]:
string_features = ['Date','Address','Species','Street','AddressNumberAndStreet','Trap','CodeSum']

X = X.drop(['NumMosquitos','Unnamed: 0','WnvPresent'],axis=1)
y = df_train['NumMosquitos']

X = X.drop(string_features,axis=1)
df_test = df_test.drop(string_features,axis=1)
df_test = df_test.drop('Id',axis=1)
#X.head()

In [80]:
X.dtypes[:20]

Block                int64
Latitude           float64
Longitude          float64
AddressAccuracy      int64
Station              int64
Tmax                 int64
Tmin                 int64
Tavg               float64
Depart             float64
DewPoint             int64
WetBulb            float64
Heat                 int64
Cool                 int64
Sunrise            float64
Sunset             float64
Depth              float64
SnowFall           float64
PrecipTotal        float64
StnPressure        float64
SeaLevel           float64
dtype: object

In [81]:
[var for var in X.columns if var not in df_test.columns]

[]

In [82]:
[var for var in df_test.columns if var not in X.columns]

[]

In [83]:
clf = RandomForestRegressor(n_estimators=100)

In [84]:
clf.fit(X,y)

RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0)

In [85]:
yhat = clf.predict(df_test)

In [86]:
y.mean(),yhat.mean()

(12.853512278697886, 11.357570011466832)

In [87]:
clf.score(df_test,yhat)

1.0

In [88]:
df_test['NumMosquitos'] = yhat

In [90]:
df_test.to_csv('aggregated_test_NumMosquito_no_strings.csv')

In [65]:
df_test.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,Lat_int,...,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,NumMosquitos
0,2,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,18.268333
1,3,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,17.746500
2,1,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,13.918333
3,4,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,17.376500
4,6,41,36,1,41.95469,-87.800991,9,6,11,41,...,64,0,11,0,29.34,29.97,9.4,18,10.4,16.566500
